In [2]:
from sklearn.manifold import TSNE
from pandas.core.frame import DataFrame
import tekore as tk
import numpy as np
import pandas as pd
from numpy.polynomial.chebyshev import chebfit,chebval
from sklearn.decomposition import PCA
import plotly.express as px

In [74]:
client_id = '4cc0d73dc6ae4e64b7318da7236f7f04'
client_secret = '3a88b3d055854dcb9cfe1a1ed8784942'
SPOTIFY_REDIRECT_URI= "http://localhost/"
app_token = tk.request_client_token(client_id, client_secret)

In [78]:
# sp = tk.Spotify(app_token)
sp.track('49sMsFUxaaGDlaKZNBfVSd').album

SimpleAlbum with fields:
  album_group = None
  album_type = 'album'
  artists = [1 x SimpleArtist(external_urls, href, id, name, type, uri)]
  available_markets = [0 x str]
  external_urls = {'spotify'}
  href = 'https://api.spotify.com/v1/albums/3Aod8ev2Xu4swkRslc26tl'
  id = '3Aod8ev2Xu4swkRslc26tl'
  images = [3 x Image(height, url, width)]
  is_playable = None
  name = 'Hijas Del Tomate'
  release_date = '2002-07-30'
  release_date_precision = 'day'
  total_tracks = 10
  type = 'album'
  uri = 'spotify:album:3Aod8ev2Xu4swkRslc26tl'

In [ ]:
def interval_vec(interval_list, title):
    dur = 0
    for i in interval_list:
        dur += i.duration
    titles = [f'average {title} interval duration', 'num of intervals']
    data= dur / len(interval_list), len(interval_list)
    return pd.Series(index = titles, data=data)

def section_vec(section_list, duration):
    loudness = 0
    tempo = 0
    key = 0
    loud_l =[]
    tempo_l = []
    key_l = []
    key_c, tempo_c, loud_c = 0,0,0
    for i in section_list:
        percentage = i.duration / duration
        loudness += percentage * i.loudness
        tempo += percentage * i.tempo
        key += percentage * i.key
        if len(loud_l):
            key_c += key_l[-1] != i.key
            tempo_c += tempo_l[-1] < i.tempo
            loud_c += loud_l[-1] < i.loudness
        loud_l.append(i.loudness)
        tempo_l.append(i.tempo)
        key_l.append(i.key)
    titles = ["average loudness", "average tempo","average key", "loudness std", "tempo std", "key std", "key change", "tempo change", "loudness change", "num sections"]
    data = loudness, tempo, key, np.std(loud_l), np.std(tempo_l), np.std(key_l), key_c /(len(section_list) -1), tempo_c /(len(section_list) -1), loud_c /(len(section_list) -1), len(section_list)
    return pd.Series(index=titles, data=data)

def segment_vec(segment_list, duration):
    last_timbre = []
    last_pitches = []
    timbre_avg = np.zeros(12)
    pitches_avg = np.zeros(12)
    matrix = np.empty((len(segment_list), 25))
    dur_vec = []
    x_vec = []
    for seg in segment_list:
        cur_p =list(seg.pitches)
        cur_t =  list(seg.timbre)
        x_vec.append(seg.start)
        dur_vec.append(seg.duration)
        matrix[-1] = [seg.loudness_max ] + cur_p + cur_t

    data = np.nan_to_num(matrix.T) @ dur_vec
    func =  chebfit(x_vec,np.nan_to_num(np.sum(matrix, axis=1)),12)
    print(f"x shape: {len(x_vec)}, sum shape: {np.sum(matrix, axis=1).shape}")
    result =  chebfit(x_vec,np.nan_to_num(np.sum(matrix, axis=1)),8)
    print(result)
    titles = ["loudness_max avg"] + [f"pitches {i} avg" for i in range(12)] +[f"ptimbre {i} avg" for i in range(12)] + [f"poly coef{i % 31} {i % 25}" for i in range(result.shape[0])] 
    
    
#     return  pd.Series(index=titles, data =list(data) + list(data_std))
    return pd.Series(index=titles , data=np.concatenate([result,data]))
                    
def song_vector(song):
    duration = song.track['duration']
    song_meta = pd.Series(index=['tempo', 'loudness'], data = [song.track['tempo'], song.track['loudness']])
    sections_vec = section_vec(song.sections, duration)
    bars_vec = interval_vec(song.bars, "bars")
    beats_vec = interval_vec(song.beats, "beats")
    # segments_vec = segment_vec(song.segments, duration)
    return pd.concat([song_meta, sections_vec,bars_vec, beats_vec])
    



def get_user_token():
# get the user token from the web
    print("copy the web address after you log in to your potify account")
    token  = tk.prompt_for_user_token(client_id, client_secret,SPOTIFY_REDIRECT_URI, scope = 'user-library-read playlist-read-private')
    return tk.Spotify(token)


def get_playlist(spotify):
    # choosing one of the users playlist
    print('\nChoose one of your playlists:\n')
    for i in range(len(spotify.followed_playlists().items)):
        print(f"{i + 1}) {spotify.followed_playlists().items[i].name} - {spotify.followed_playlists().items[i].tracks.total}")
    playlist_idx = int(input("")) - 1
    return spotify.playlist_items(spotify.followed_playlists().items[playlist_idx].id, as_tracks=False).items

def get_track_feat(id):
    audio_feat = spotify.track_audio_features(id)
    return pd.Series({"acousticness":audio_feat.acousticness,
                     "danceability" :audio_feat.danceability,
                     "energy":audio_feat.energy,
                     "liveness": audio_feat.liveness,
                     "loudness": audio_feat.loudness,
                     "speechiness": audio_feat.speechiness,
                     "tempo": audio_feat.tempo,
                     "valence": audio_feat.valence})

def get_songs_data(tracks)-> pd.DataFrame: 
    songs_s = []
    # songs_id = [i.track.id for i in tracks]
    artists_s = pd.Series()
    for i in range(len(tracks)):
        print(f"idx: {i} length list: {len(songs_s)} ")
        if not i + 1 % 100:
            time.sleep(15)
            print("sleeping!")
            pd.concat(songs_s, axis=1).T.to_csv("test_df.csv")
        track = tracks[i]
        song = track.track
        song_data = song_vector(spotify.track_audio_analysis(song.id))
        song_f = get_track_feat(song.id)
        song_date = pd.Series(data=[song.album.release_date.split('-')[0]], index=["release year"])
        artists_s = pd.concat([artists_s,pd.Series(data=["|".join([i.name for i in song.artists])],index=[song.name + '-' + song.artists[0].name])])
        song_data = pd.concat([song_f, song_data, song_date]).rename(song.name + '-' + song.artists[0].name )
        songs_s.append(song_data)
    add = artists_s.str.get_dummies()
    result = pd.concat(songs_s, axis=1)
    return pd.concat([result.T, add], axis=1)
 

In [ ]:

# No need to run this!
# client_id = '4cc0d73dc6ae4e64b7318da7236f7f04'
# client_secret = '3a88b3d055854dcb9cfe1a1ed8784942'
# SPOTIFY_REDIRECT_URI= "http://localhost/"
# app_token = tk.request_client_token(client_id, client_secret)
# spotify = get_user_token()
# tracks = []
# for i in range(5):
#     tracks = tracks + list(get_playlist(spotify))
# df = get_songs_data(tracks)
# df.T.to_csv("test_df.csv")

In [39]:
df= pd.read_csv("spotify/test_df.csv", index_col=0).T
df

,acousticness,danceability,energy,liveness,loudness,speechiness,tempo,valence,tempo,loudness,...,Yoni Rechter,Zehava Ben,אז. עכשיו.,אריסה,אתניקס,גלי עטרי,הכוכב הבא,חלב ודבש,שחר חסון,שרי
האם האם האם-Gidi Gov,0.34500,0.414,0.648,0.3550,-7.239,0.0546,120.208,0.4020,120.208,-7.239,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
הגננים היום עצובים-Yehudit Ravitz,0.26500,0.522,0.407,0.0914,-11.926,0.0332,104.159,0.3180,104.159,-11.926,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
הוא-Noa,0.86100,0.301,0.279,0.9710,-13.746,0.0407,140.794,0.0969,140.794,-13.746,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
הוא הולך ממך-Yehudit Ravitz,0.39600,0.191,0.310,0.0811,-16.038,0.0340,153.235,0.1320,153.235,-16.038,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
הולכת ממך-Sharon Haziz,0.13400,0.627,0.840,0.0854,-8.187,0.0403,96.994,0.7240,96.994,-8.187,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
נעמיד פני יתומים-Yehudit Ravitz,0.00747,0.466,0.464,0.2200,-11.048,0.0264,78.123,0.6450,78.123,-11.048,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
נשים רוקדות-Chava Alberstein,0.51600,0.587,0.548,0.9650,-8.898,0.3680,100.273,0.7440,100.273,-8.898,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
סיגליות-David Broza,0.57900,0.572,0.545,0.2270,-10.282,0.0316,125.699,0.5770,125.699,-10.282,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
סליחות-Yehudit Ravitz,0.86800,0.373,0.174,0.1950,-14.733,0.0302,99.799,0.5140,99.799,-14.733,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
#Normalizing the needed columns (other columns are one hot vector or year)
df.iloc[:,:24] = (df.iloc[:,:24]-df.iloc[:,:24].mean())/df.iloc[:,:24].std()
# dropping columns which did not contribute
dff =  df.drop(['loudness change','key change', 'tempo change'], axis=1)

In [41]:
# Doing PCA
pca = PCA(n_components=2)
pca.fit(dff)
X = pca.transform(dff)
# creating DF for the visualization
rdf = pd.DataFrame(data=X, index=df.index).reset_index()
rdf['release year'] = df['release year'].tolist()

fig = px.scatter(rdf, x=0, y=1, hover_data=["index",'release year' ],color='release year')
fig.update_xaxes(title_text=f"PC 1 ({round(pca.explained_variance_ratio_[0],2)})")
fig.update_yaxes(title_text=f"PC 2 ({round(pca.explained_variance_ratio_[1],2)})")
fig.show()

In [32]:
#T-SNE analysis
X_embedded = TSNE(n_components=2).fit_transform(dff)
rdf = pd.DataFrame(data=X_embedded, index=df.index).reset_index()
rdf['release year'] = df['release year'].tolist()
fig = px.scatter(rdf, x=0, y=1, hover_data=["index", "release year"], color="release year",)

fig.show()

In [33]:
# Now lets do it all again w/o the year
dff =  dff.drop(['release year'], axis=1)

In [35]:
# Doing PCA
pca = PCA(n_components=2)
pca.fit(dff)
X = pca.transform(dff)
# creating DF for the visualization
rdf = pd.DataFrame(data=X, index=df.index).reset_index()
rdf['release year'] = df['release year'].tolist()

fig = px.scatter(rdf, x=0, y=1, hover_data=["index",'release year' ], color='release year')
fig.update_xaxes(title_text=f"PC 1 ({round(pca.explained_variance_ratio_[0],2)})")
fig.update_yaxes(title_text=f"PC 2 ({round(pca.explained_variance_ratio_[1],2)})")
fig.show()

In [36]:
#T-SNE analysis
X_embedded = TSNE(n_components=2).fit_transform(dff)
rdf = pd.DataFrame(data=X_embedded, index=df.index).reset_index()
rdf['release year'] = df['release year'].tolist()
fig = px.scatter(rdf, x=0, y=1, hover_data=["index", "release year"], color="release year")
fig.show()

In [13]:
%matplotlib inline
df_subset['tsne-2d-one'] = t_res[:,0]
df_subset['tsne-2d-two'] = t_res[:,1]
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue="y",
    palette=sns.color_palette("hls", 10),
    data=df_subset,
    legend="full",
    alpha=0.3
)

NameError: name 'df_subset' is not defined